In [1]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.export import to_csv
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
#file handling
from pathlib import Path
import os

In [2]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [3]:
FILE_PREFIX = "HL_428_C_E"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"
LDA_DIR = RESULT_DIR + "/" + "lda_dim_reduction"
REG_DIR = RESULT_DIR + "/" + "regularized_models/"
TREE_DIR = RESULT_DIR + "/" + "tree_based_models/"

## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [4]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [5]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [6]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [7]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [8]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [9]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [10]:
SCORING = ['accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [11]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_control, path_etoposide], ['Control', 'Etoposide'])
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [12]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [13]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement Stratified KFold CV

In [14]:
path_in = PREP_OUT

data = load_data(path_in)

X = data.drop(columns=["label", "file"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

stk = StratifiedKFold(n_splits=10)

## LDA Dimensionality Reduction
### Baseline with LDA alone

In [15]:
clf = LinearDiscriminantAnalysis()
result = cross_validate(clf, X, y, cv=stk, scoring=SCORING, return_train_score=True)
to_csv(result, path=LDA_DIR + "/lda", scoring=SCORING, param_opt=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Feature Selection with PCA followed by LDA

In [16]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/pca_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Inte

### Feature Selection with Nonnegative Matric Factorization followed by LDA

In [17]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/nmf_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


### Feature Selection with Feature Agglomeration followed by LDA

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Inte

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[CV 1/10] END pca__n_components=1; accuracy: (train=0.711, test=0.600) f1: (train=0.662, test=0.667) total time=   1.5s
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[CV 2/10] END pca__n_components=1; accuracy: (train=0.700, test=0.733) f1: (train=0.657, test=0.692) total time=   1.5s
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated

...

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[CV 8/10] END pca__n_components=1; accuracy: (train=0.711, test=0.650) f1: (train=0.665, test=0.533) total time=   1.5s
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[CV 9/10] END pca__n_components=1; accuracy: (train=0.702, test=0.767) f1: (train=0.655, test=0.696) total time=   1.4s
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated

/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[CV 10/10] END pca__n_components=1; accuracy: (train=0.704, test=0.733) f1: (train=0.655, test=0.680) total time=   1.5s
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecate

### Feature Selection with Feature Agglomeration followed by LDA

In [18]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/fa_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 8 candidates, totalling 80 fits
[CV 1/10] END agglo__n_clusters=5; accuracy: (train=0.869, test=0.650) f1: (train=0.862, test=0.632) total time=   0.2s
[CV 2/10] END agglo__n_clusters=5; accuracy: (train=0.852, test=0.833) f1: (train=0.846, test=0.828) total time=   0.2s
[CV 3/10] END agglo__n_clusters=5; accuracy: (train=0.861, test=0.717) f1: (train=0.857, test=0.679) total time=   0.2s
[CV 4/10] END agglo__n_clusters=5; accuracy: (train=0.846, test=0.867) f1: (train=0.839, test=0.862) total time=   0.2s
[CV 5/10] END agglo__n_clusters=5; accuracy: (train=0.848, test=0.850) f1: (train=0.842, test=0.842) total time=   0.2s
[CV 6/10] END agglo__n_clusters=5; accuracy: (train=0.826, test=0.950) f1: (train=0.816, test=0.949) total time=   0.2s
[CV 7/10] END agglo__n_clusters=5; accuracy: (train=0.839, test=0.900) f1: (train=0.832, test=0.893) total time=   0.2s
[CV 8/10] END agglo__n_clusters=5; accuracy: (train=0.846, test=0.833) f1: (train=0.839, test=0.828

### Feature Selection with PeakPicker followed by LDA

In [19]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/peak_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 10 folds for each of 15 candidates, totalling 150 fits
[CV 1/10] END peaks__min_dist=10; accuracy: (train=0.959, test=0.783) f1: (train=0.958, test=0.755) total time=   0.0s
[CV 2/10] END peaks__min_dist=10; accuracy: (train=0.950, test=0.983) f1: (train=0.949, test=0.983) total time=   0.0s
[CV 3/10] END peaks__min_dist=10; accuracy: (train=0.963, test=0.833) f1: (train=0.962, test=0.815) total time=   0.0s
[CV 4/10] END peaks__min_dist=10; accuracy: (train=0.948, test=1.000) f1: (train=0.947, test=1.000) total time=   0.0s
[CV 5/10] END peaks__min_dist=10; accuracy: (train=0.948, test=0.950) f1: (train=0.947, test=0.947) total time=   0.0s
[CV 6/10] END peaks__min_dist=10; accuracy: (train=0.950, test=1.000) f1: (train=0.949, test=1.000) total time=   0.0s
[CV 7/10] END peaks__min_dist=10; accuracy: (train=0.943, test=0.950) f1: (train=0.941, test=0.947) total time=   0.0s
[CV 8/10] END peaks__min_dist=10; accuracy: (train=0.952, test=0.900) f1: (train=0.951, test=0.897) tota

## Regularized Models
### Logistic Regression L1 Penalty

In [20]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
    penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END logreg__C=0.01; accuracy: (train=0.913, test=0.717) f1: (train=0.905, test=0.638) total time=   0.1s
[CV 2/10] END logreg__C=0.01; accuracy: (train=0.900, test=0.850) f1: (train=0.891, test=0.824) total time=   0.1s
[CV 3/10] END logreg__C=0.01; accuracy: (train=0.909, test=0.817) f1: (train=0.901, test=0.784) total time=   0.1s
[CV 4/10] END logreg__C=0.01; accuracy: (train=0.889, test=0.950) f1: (train=0.878, test=0.947) total time=   0.1s
[CV 5/10] END logreg__C=0.01; accuracy: (train=0.896, test=0.900) f1: (train=0.887, test=0.889) total time=   0.1s
[CV 6/10] END logreg__C=0.01; accuracy: (train=0.902, test=0.917) f1: (train=0.892, test=0.915) total time=   0.1s
[CV 7/10] END logreg__C=0.01; accuracy: (train=0.893, test=0.967) f1: (train=0.882, test=0.966) total time=   0.1s
[CV 8/10] END logreg__C=0.01; accuracy: (train=0.898, test=0.917) f1: (train=0.888, test=0.909) total time=   0.1s
[CV 9/10] END log

### Logistic Regression L2 Penalty

In [21]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV 1/10] END logreg__C=1e-05; accuracy: (train=0.920, test=0.667) f1: (train=0.915, test=0.583) total time=   0.0s
[CV 2/10] END logreg__C=1e-05; accuracy: (train=0.902, test=0.917) f1: (train=0.893, test=0.909) total time=   0.0s
[CV 3/10] END logreg__C=1e-05; accuracy: (train=0.909, test=0.817) f1: (train=0.903, test=0.784) total time=   0.0s
[CV 4/10] END logreg__C=1e-05; accuracy: (train=0.896, test=0.967) f1: (train=0.887, test=0.966) total time=   0.0s
[CV 5/10] END logreg__C=1e-05; accuracy: (train=0.904, test=0.917) f1: (train=0.896, test=0.909) total time=   0.0s
[CV 6/10] END logreg__C=1e-05; accuracy: (train=0.900, test=0.950) f1: (train=0.892, test=0.947) total time=   0.0s
[CV 7/10] END logreg__C=1e-05; accuracy: (train=0.896, test=0.967) f1: (train=0.887, test=0.966) total time=   0.0s
[CV 8/10] END logreg__C=1e-05; accuracy: (train=0.907, test=0.867) f1: (train=0.900, test=0.857) total time=   0.0s
[CV 9/10]

### Linear SVM L1 Penalty

In [22]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=15000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 2/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 3/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 4/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 5/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 6/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 7/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 8/10] END svm__C=0.001; accuracy: (train=0.500, test=0.500) f1: (train=0.000, test=0.000) total time=   0.0s
[CV 9/10] END svm__C=0.001; accur

### Linear SVM L2 Penalty

In [23]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV 1/10] END svm__C=1e-05; accuracy: (train=0.937, test=0.717) f1: (train=0.933, test=0.653) total time=   0.0s
[CV 2/10] END svm__C=1e-05; accuracy: (train=0.917, test=0.933) f1: (train=0.911, test=0.929) total time=   0.0s
[CV 3/10] END svm__C=1e-05; accuracy: (train=0.933, test=0.817) f1: (train=0.930, test=0.792) total time=   0.0s
[CV 4/10] END svm__C=1e-05; accuracy: (train=0.922, test=0.983) f1: (train=0.917, test=0.983) total time=   0.0s
[CV 5/10] END svm__C=1e-05; accuracy: (train=0.922, test=0.933) f1: (train=0.917, test=0.929) total time=   0.0s
[CV 6/10] END svm__C=1e-05; accuracy: (train=0.919, test=0.967) f1: (train=0.913, test=0.966) total time=   0.0s
[CV 7/10] END svm__C=1e-05; accuracy: (train=0.922, test=0.967) f1: (train=0.917, test=0.966) total time=   0.0s
[CV 8/10] END svm__C=1e-05; accuracy: (train=0.922, test=0.883) f1: (train=0.917, test=0.877) total time=   0.0s
[CV 9/10] END svm__C=1e-05; accur

## Tree-based Models
### Basic Decision Tree

In [24]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/decision_tree", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.700) f1: (train=1.000, test=0.690) total time=   0.6s
[CV 2/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.867) f1: (train=1.000, test=0.862) total time=   0.7s
[CV 3/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.833) f1: (train=1.000, test=0.848) total time=   0.7s
[CV 4/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.900) f1: (train=1.000, test=0.903) total time=   0.6s
[CV 5/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.983) f1: (train=1.000, test=0.983) total time=   0.8s
[CV 6/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.950) f1: (train=1.000, test=0.951) total time=   0.7s
[CV 7/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.950) f1: (train=1.000, test=0.949) total time=   0.8s
[CV 8/10] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.867) f1: (train=1.000, test=0.857) total time=   0.7s
[CV 9/10] E

### Random Forest

In [25]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/random_forest", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 1/10] END colsample_bytree=0.01; accuracy: (train=0.983, test=0.817) f1: (train=0.983, test=0.800) total time=   0.2s
[CV 2/10] END colsample_bytree=0.01; accuracy: (train=0.976, test=0.967) f1: (train=0.975, test=0.966) total time=   0.2s
[CV 3/10] END colsample_bytree=0.01; accuracy: (train=0.983, test=0.900) f1: (train=0.983, test=0.893) total time=   0.1s
[CV 4/10] END colsample_bytree=0.01; accuracy: (train=0.981, test=0.950) f1: (train=0.981, test=0.949) total time=   0.2s
[CV 5/10] END colsample_bytree=0.01; accuracy: (train=0.978, test=1.000) f1: (train=0.977, test=1.000) total time=   0.3s
[CV 6/10] END colsample_bytree=0.01; accuracy: (train=0.983, test=1.000) f1: (train=0.983, test=1.000) total time=   0.1s
[CV 7/10] END colsample_bytree=0.01; accuracy: (train=0.978, test=1.000) f1: (train=0.977, test=1.000) total time=   0.2s
[CV 8/10] END colsample_bytree=0.01; accuracy: (train=0.980, test=0.933) f1: (train

### Gradient-boosted Decision Trees

In [26]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=stk, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/gbdt", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 1/10] END learning_rate=0.01; accuracy: (train=0.983, test=0.800) f1: (train=0.983, test=0.769) total time=   0.3s
[CV 2/10] END learning_rate=0.01; accuracy: (train=0.987, test=0.933) f1: (train=0.987, test=0.929) total time=   0.3s
[CV 3/10] END learning_rate=0.01; accuracy: (train=0.996, test=0.933) f1: (train=0.996, test=0.933) total time=   0.3s
[CV 4/10] END learning_rate=0.01; accuracy: (train=0.976, test=0.950) f1: (train=0.976, test=0.951) total time=   0.3s
[CV 5/10] END learning_rate=0.01; accuracy: (train=0.987, test=0.967) f1: (train=0.987, test=0.968) total time=   0.3s
[CV 6/10] END learning_rate=0.01; accuracy: (train=0.987, test=0.967) f1: (train=0.987, test=0.967) total time=   0.3s
[CV 7/10] END learning_rate=0.01; accuracy: (train=0.981, test=0.950) f1: (train=0.982, test=0.949) total time=   0.3s
[CV 8/10] END learning_rate=0.01; accuracy: (train=0.987, test=0.933) f1: (train=0.987, test=0.931) tota